In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
import os

In [ ]:
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"
os.environ["OPENAI_MODEL_NAME"] = "llama3-70b-8192"
os.environ["SERPER_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [ ]:
from crewai_tools import BaseTool

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=False
)

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)

Here are three personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng:

**Email Draft 1:**

Subject: Congratulations on Your Recent Product Launches!

Dear Andrew,

I wanted to extend my warm congratulations on the successful launch of your new Data Analytics Professional Certificate on Coursera. This is a significant milestone in DeepLearningAI's mission to make AI education accessible to everyone. I'm impressed by your commitment to providing practical and accessible education in AI and ML.

Our solutions at CrewAI can support your mission by providing scalable infrastructure, data analytics, and AI-powered tools. We believe that our customized solutions can cater to your platform's specific needs, helping you to further democratize AI education.

I'd love to discuss how we can help you achieve your goals. Please let me know if you're open to exploring this further.

Best regards,
[Your Name]

**Email Draft 2:**

Subject: Enhancing AI Education with Personalized Solutions

Hi Andrew,

I've been following DeepLearningAI's progress, and I'm thrilled to see the impact you're making in the AI education space. Your "AI for Everyone" and "Generative AI for Everyone" courses are truly game-changers.

At CrewAI, we're passionate about supporting innovative companies like yours. Our solutions can help you optimize your platform's performance, providing a seamless learning experience for your users. We'd love to discuss how our AI-powered tools can support your mission to make AI education accessible to everyone.

Would you be open to a call to explore this further?

Best regards,
[Your Name]

**Email Draft 3:**

Subject: Unlocking the Power of AI in Education

Hi Andrew,

I came across your article on "Letters from Andrew Ng" and was struck by your vision for the future of AI product management. Your commitment to empowering individuals with AI skills is truly inspiring.

At CrewAI, we share your passion for democratizing AI education. Our solutions can help you unlock the full potential of AI in education, providing a personalized learning experience for your users. We believe that our customized solutions can support your mission to make AI education accessible to everyone.

I'd love to discuss how we can support your goals. Please let me know if you're interested in exploring this further.

Best regards,
[Your Name]

Each draft is tailored to resonate with DeepLearningAI's company culture and values, demonstrating a deep understanding of their business and needs. The tone is engaging, professional, and aligned with DeepLearningAI's corporate identity.